In [34]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline

### Static Variables

In [35]:
linefiles = os.listdir("bus_data/clean_data2")
lines = []
for line in linefiles:
    lines.append(line.replace(".csv",""))
    


### Extracting Data

### Read in External Data

In [36]:
schedule = pd.read_csv("bus_data/dublinbus_scheduledData2013csv.csv", low_memory=False, encoding="ISO-8859-1")

In [37]:
# Dropping null values - some in our dataframe are marked as 'null' (string) for some reason?
schedule = schedule.dropna()
schedule = schedule[schedule.stop_id != 'null']

### Exploring schedule

In [38]:
# Extracting StopID from stop_id

def get_stop(string):
    var = string[-4:]
    return var

schedule["StopID"] = schedule['stop_id'].apply(lambda x: get_stop(x))

In [39]:
# Extracting LineID from trip_id
schedule["LineID"] = schedule["trip_id"].str.extract('\-(.*?)\-')

C:\Users\eogha\Anaconda3\envs\comp47350\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


In [40]:
# Convert StopIDs back to int
# df['StopID'] = df['StopID'].astype('int64')
schedule['StopID'] = schedule['StopID'].astype('int64')

In [41]:
# Extracting direction from trip_id
def get_direction(string):
    var = string[-1:]
    return var

schedule["Direction"] = schedule['trip_id'].apply(lambda x: get_direction(x))

In [42]:
schedule = schedule[(schedule.Direction == "I") | (schedule.Direction == "O")]    

In [43]:
schedule.head()

,trip_id,arrival_time,departure_time,stop_id,Name without locality,Name,Lat,Long,stop_sequence,stop_headsign,shape_dist_traveled,StopID,LineID,Direction
0,1.1598.0-33A-b12-1.451.I,16:15:00,16:15:00,8240DB003813,Footbridge,Skerries Road,53.591377,-6.156176,1,Swords,0.000000,3813,33A,I
1,1.1598.0-33A-b12-1.451.I,16:15:39,16:15:39,8240DB003814,Barnageeragh Road,Balbriggan Rd,53.585544,-6.148605,2,Swords,827.070328,3814,33A,I
2,1.1598.0-33A-b12-1.451.I,16:17:11,16:17:11,8240DB003815,Barnageeragh Road,North Cliff Heights,53.581935,-6.125247,3,Swords,2746.649492,3815,33A,I
3,1.1598.0-33A-b12-1.451.I,16:17:22,16:17:22,8240DB003816,Mourne Drive,North Cliff Heights,53.580471,-6.127622,4,Swords,2973.900755,3816,33A,I
4,1.1598.0-33A-b12-1.451.I,16:17:44,16:17:44,8240DB003566,Selskar Court,Town Parks Rd,53.578067,-6.125055,5,Swords,3447.155571,3566,33A,I


In [44]:
# Converting Direction to JourneyPatternID
schedule["JourneyPatternID"] = schedule['Direction'].apply(lambda x: "1001" if x == "I" else "0001")
schedule['JourneyPatternID'] = schedule['JourneyPatternID'].astype('int64')

In [45]:
# dropping irrelevant columns
for column in ['trip_id', 'stop_id', 'arrival_time', 'departure_time', 'shape_dist_traveled', 'Direction']:
        schedule = schedule.drop(column, 1)

In [46]:
merge_schedule = schedule[['Name without locality','Name','Lat','Long','StopID']]

## Single Extraction
    

In [33]:
temp = schedule[(schedule['LineID'] == "68")]
temp.JourneyPatternID.value_counts()

1       3324
1001    3208
Name: JourneyPatternID, dtype: int64

In [26]:
schedule[(schedule['LineID'] == "68")].stop_headsign.value_counts()

Hawkins St      2948
Greenogue       1924
Newcastle       1400
Conyngham Rd     260
Name: stop_headsign, dtype: int64

In [47]:
line = lines[96]

#Iterate through line data files
# for line in lines:
df = pd.read_csv("bus_data/clean_data2/" + line + ".csv", low_memory=False, header=None)
df.columns = ["Timestamp", "LineID", "JourneyPatternID", "TimeFrame", 
              "VehicleJourneyID", "Lon", "Lat", "VehicleID", "StopID", 
              "AtStop", "HumanTime", "Day", "Hour", "JourneyGroup", "Runtime"]

line

'68'

In [48]:
df = df.drop(["Timestamp", "TimeFrame", "VehicleJourneyID", "VehicleID", 
              "AtStop", "HumanTime", "Day", "Hour", "JourneyGroup", "Runtime"], 1)
df.LineID = df.LineID.astype("object")
df['StopID'] = df['StopID'].astype('str')

df.StopID = df.StopID.astype("int64")

new_df = df['StopID'].drop_duplicates()

new_df = pd.DataFrame(new_df)

In [49]:

# merged_df = pd.merge(new_df, schedule, how='inner', on=['StopID'])
merged_df = pd.merge(new_df, schedule, how='inner', on=['StopID'])
merged_df = merged_df.drop_duplicates()


In [50]:
line_df = merged_df[(merged_df.LineID == line)]
line_df[(line_df.LineID == line)].stop_headsign.value_counts().index
# tdf = line.drop_duplicates(subset=['stop_headsign'], keep='last')

Index(['Greenogue', 'Newcastle'], dtype='object')

In [51]:
a = line_df[(line_df.LineID == line)].JourneyPatternID.value_counts().index
a

Int64Index([1], dtype='int64')

In [52]:
rowlists = []

In [53]:
rowlists.append([jp1,hs1,line])
rowlists.append([jp2,hs2,line])

NameError: name 'jp1' is not defined

In [128]:
ldf = pd.DataFrame(rowlists,columns=['JourneyPatternID','Headsign','LineID'])

# Find Route Subsections  - Looped

In [138]:
rowlists = []

### Read in Line Data

In [145]:
# line = lines[70]
for i in range(97, len(lines)):
    line = lines[i]
    print(line)
    #Iterate through line data files
    # for line in lines:
    df = pd.read_csv("bus_data/clean_data2/" + line+ ".csv", low_memory=False, header=None)
    df.columns = ["Timestamp", "LineID", "JourneyPatternID", "TimeFrame", 
                  "VehicleJourneyID", "Lon", "Lat", "VehicleID", "StopID", 
                  "AtStop", "HumanTime", "Day", "Hour", "JourneyGroup", "Runtime"]
    df = df.drop(["Timestamp", "TimeFrame", "VehicleJourneyID", "VehicleID", 
                  "AtStop", "HumanTime", "Day", "Hour", "JourneyGroup", "Runtime"], 1)
    df.LineID = df.LineID.astype("object")
    df['StopID'] = df['StopID'].astype('str')
    df = df[df.StopID != 'null']

    df.StopID = df.StopID.astype("int64")

    new_df = df['StopID'].drop_duplicates()

    new_df = pd.DataFrame(new_df)

    # merged_df = pd.merge(new_df, schedule, how='inner', on=['StopID'])
    merged_df = pd.merge(new_df, schedule, how='inner', on=['StopID'])

    merged_df = merged_df.drop_duplicates()

    line_df = merged_df[(merged_df.LineID == line)]
    
    if len(line_df[(line_df.LineID == line)].stop_headsign.value_counts()) == 1:
        hs1 = line_df[(line_df.LineID == line)].stop_headsign.value_counts().index[0]
        jp1 = line_df[(line_df.LineID == line)].JourneyPatternID.value_counts().index[0]
        rowlists.append([jp1,hs1,line])
    elif len(line_df[(line_df.LineID == line)].stop_headsign.value_counts()) == 0:
        continue
    else:
        hs1 = line_df[(line_df.LineID == line)].stop_headsign.value_counts().index[0]
        hs2 = line_df[(line_df.LineID == line)].stop_headsign.value_counts().index[1]
        # tdf = line.drop_duplicates(subset=['stop_headsign'], keep='last')


        line_df[(line_df.LineID == line)].JourneyPatternID.value_counts()

        jp1 = line_df[(line_df.LineID == line)].JourneyPatternID.value_counts().index[0]
        jp2 = line_df[(line_df.LineID == line)].JourneyPatternID.value_counts().index[1]

        rowlists.append([jp1,hs1,line])
        rowlists.append([jp2,hs2,line])

68A
69
69X
7
70
747
75
76
76A
77A
79
79A
7B
7D
8
83
83A
84
84A
84X
9
PP07


In [146]:
len(rowlists)

224

In [147]:
ldf = pd.DataFrame(rowlists,columns=['JourneyPatternID','Headsign','LineID'])


In [148]:
ldf

,JourneyPatternID,Headsign,LineID
0,1,Sandymount,1
1,1001,O'Connell Street,1
2,1001,Station Rd,102
3,1,Airport,102
4,1001,Clontarf Road,104
5,1,Shanard Road,104
6,1,Sandyford I.E.,11
7,1001,City Centre,11
8,1001,Loughlinstown,111
9,1,Dun Laoghaire,111


In [149]:
ldf.to_csv("68missing.csv")